In [3]:
library(dplyr)
library(survival)
library(moonBook)
library(survival)
library(survminer)

## 1. No competing risk

### 1. 데이터 읽기

In [4]:
data = data.frame(data.table::fread("파일명"))

### 2. 데이터 전처리

#### 2.1 날짜형 수정

In [6]:
as.Date(as.character(data1$"문자형 변수명"),format="%Y%m%d")

#### 2.2 범주화
> 기준이 다른 변수들을 한번에 쉽게 코드화 할 수 있는지 공부하기

In [8]:
data$"변수명" = ifelse(data$"변수명" == 1 , "YES","NO")
data$"변수명" = factor(data$"변수명", levels = c("YES","NO"))

### 3. Table 1

In [ ]:
t1 = tableone::CreateTableOne(vars=c("변수명 집합"),strata = "기준 변수",data=data,test=F, addOverall = T)
# write.csv(print(t1,smd=T),"AnyC_DTH_t1.csv")

### 4. 중도 절단 고려한 생존기간 설정

In [11]:
# library(survival)

Surv(data$"생존 시간 변수", data$"타겟 변수" == 1)

summary(Surv(data$"생존 시간 변수", data$"타겟 변수" == 1))

# 특정 시간 생존률
summary(survfit(Surv(data$"생존 시간 변수", data$"타겟 변수" == 1) ~ "기준 변수", data = data), c(1,3)*365.25)

### 5. Kaplan-Meiere function

`-` 1년 단위 설정

In [ ]:
xscale = "d_y"
xtrans <- switch(xscale,
                 d_m = 12/365.25,
                 d_y = 1/365.25,
                 m_d = 365.25/12,
                 m_y = 1/12,
                 y_d = 365.25,
                 y_m =12,
                 1)

breaks = seq(0,1250,730)


`-` 그래프

In [ ]:
library(survminer)

ggsurvplot(survfit(Surv(data$"생존 시간 변수", data$"타겟 변수" == 1) ~ "기준 변수",data=data),
            data= data,
            fun="event",
            size=1,
            palette = c("#2E9FDF","#df2e46"),
            censor=F,
            conf.int=T,
            surv.scale = "percent",
            pval = T,
            pval.coord = c(0.2, 0.2),
            xscale = "d_y",
            break.x.by = 365.25*2,
            xlim = c(0,3700),
            xlab = "Years",
            ylab = "Cumulative incidence probability (%)",
            legend.labs = c("대조군", "실험군"),
            legend.title = "Group",
            legend = c(0.1, 0.85),
            risk.table = "nrisk_cumevents",
            risk.table.col="기준 변수",
            risk.table.height = 0.2)
            # fontsize = 3,
            # ggtheme = theme_bw())

### 5. Various statistical metrics

#### 5.1 Hazard ratio
> Adjusted와 conditional (Unadjusted와 marginal)은 자세히 공부하면 상호교환 가능 단어가 아니라는 논문이 있지만 여기서는 같은 의미로 작성함.

`-` Unadjusted HR

In [ ]:
# library(moonBook)

out = mycph(Surv(data$"생존 시간 변수", data$"타겟 변수" == 1) ~ ., data=data)

HRplot(out)

coef = out

paste0(sprintf("%.2f",coef[row,1]),"(", sprintf("%.2f",coef[row,2]),"-", sprintf("%.2f",coef[row,3]),")")

`-` Adjusted HR

In [ ]:
out2 = coxph(Surv(data$"생존 시간 변수", data$"타겟 변수" == 1) ~ ., data=data)

HRplot(out2,show.CI=T)

coef = summary(out2)$coefficients

coef[row,2]

exp(coef[row,1]-1.96*coef[row,3])

exp(coef[row,1]+1.96*coef[row,3])

paste0(round(coef[row,2],2),"(", round(exp(coef[row,1]-1.96*coef[row,3]),2),"-", round(exp(coef[row,1]+1.96*coef[row,3]),2),")")

#### 5.2 p for interaction

`-` Unadjusted interaction

In [ ]:
fit1 = coxph(Surv(data$"생존 시간 변수", data$"타겟 변수" == 1) ~ "변수1" + "변수2", data = data)
fit2 = coxph(Surv(data$"생존 시간 변수", data$"타겟 변수" == 1) ~ "변수1" + "변수2" + "변수1":"변수2", data = data)
interaction = anova(fit1, fit2)
interaction$`Pr(>|Chi|)`[2]

`-` Adjusted interaction

In [ ]:
fit1 = coxph(Surv(data$"생존 시간 변수", data$"타겟 변수" == 1) ~ ., data = data)
fit2 = coxph(Surv(data$"생존 시간 변수", data$"타겟 변수" == 1) ~ . + AnyC_:Sex_, data = data)
interaction = anova(a, b)
interaction$`Pr(>|Chi|)`[2]

#### 5.3 10000 person-year

In [ ]:
py10000 = sum(data[data[,colName]==con,]$"타겟 변수") / (sum(as.numeric(data[data[,colName]==con,]$"생존 시간 변수")) / 365)*10000

#### 5.4 SMD

In [ ]:
tableone::ExtractSmd(tableone::CreateCatTable(c("변수명 집합"),"기준 변수",data=data))

## 2. Competing risk

`-` 경쟁 위험은 타겟 별 원인 변수가 필요함  
`-` library(cmprsk) crr() 함수

In [ ]:
library(cmprsk)

`-` Subdistribution hazard ratio

In [ ]:
# 모델에 들어갈 변수 cbind
cov.mat = cbind(data$"변수명")

crr.1 = crr(data$"생존기간 변수",data$"원인 별 변수",cov.mat, failcode = "관심 원인 값",cencode="중도 절단 값"))

Subdistribution_coef = summary(crr.1)$coef

paste(Subdistribution_coef[1,2], "(", exp(Subdistribution_coef[1,1]-1.96*Subdistribution_coef[1,3]),"-", exp(Subdistribution_coef[1,1]+1.96*Subdistribution_coef[1,3]),")" )

`-` cause‐specific hazard ratio

In [ ]:
coxph.1 = coxph(Surv(data$"생존기간 변수", data$"원인 별 변수" == "관심 원인 값") ~ data$"기준 변수")

cause_coef = summary(crr.1)$coef

paste(cause_coef[1,2], "(", exp(cause_coef[1,1]-1.96*cause_coef[1,3]),"-", exp(cause_coef[1,1]+1.96*cause_coef[1,3]),")" )

`-` Cumulative incidence function
>  정리 필요

In [ ]:
data$TS = Surv(data$survDt, data$isDTH == 1)

# summary(data$TS)

# summary(survfit(TS~AnyC_,data=data), c(1,3)*365.25)

TSCol = grep("TS",names(data))

cif1 <- cuminc(ftime=data$"생존기간 변수",fstatus=data$"원인 별 변수", data$"기준 변수", cencode="중도 절단 값") 

plot(survfit(Surv(data$"생존시간변수", data$isDTH == 1) ~ "기준 변수", data=data)$time, 1 - survfit(Surv(data$"생존시간변수", data$isDTH == 1) ~ "기준 변수",data=data)$surv, ylab="Probability")

lines(cif1$`"기준 변수1" 1`$time, cif1$`"기준 변수1" 1`$est, type="l", ylim=c(0,3), xlab="Survival time (days)",ylab="Probability", lwd = 3, lty=1,col="black") 
lines(cif1$`"기준 변수2" 1`$time, cif1$`"기준 변수2" 1`$est, type="l", ylim=c(0,0.3), xlab="Survival time (days)",ylab="Probability", lwd = 3, lty=1,col="orange")

lines(cif1$`"기준 변수1" 2`$time, cif1$`"기준 변수1" 2`$est, type="l", ylim=c(0,0.3), xlab="Survival time (days)",ylab="Probability", lwd = 3, lty=1,col="blue") 
lines(cif1$`"기준 변수2" 2`$time, cif1$`"기준 변수2" 2`$est, type="l", ylim=c(0,0.3), xlab="Survival time (days)",ylab="Probability", lwd = 3, lty=1,col="red") 

title("Figure 1. Cumulative Incidence functions")
legend("topleft", legend = c("All-cause death (1-KM)/Sum of two CIFs","Top5", "others"), lty = c(1,1,1), col = c("black","orange","red"), bty="n") 